# Testing basic tools in documentation


[x] Add    
[x] Get


In [ ]:
import molsysmt as msm

In [ ]:
item = msm.convert('1L2Y', 'mmcif.PDbxContainers.DataContainer')

In [ ]:
molsys = msm.convert('1L2Y')

In [ ]:
item.exists('entry')

# entity info

In [ ]:
item.getObj('entity')

In [ ]:
item.getObj('entity_poly')

In [ ]:
item.getObj('entity_poly_seq')

In [ ]:
item.getObj('pdbx_poly_seq_scheme')

In [ ]:
datacategory_atom_site = item.getObj('atom_site')

In [ ]:
aa = { jj:ii for ii,jj in enumerate(datacategory_atom_site.getAttributeList())}

In [ ]:
aa

In [ ]:
data = datacategory_atom_site.data

In [ ]:
data[1][15]

In [ ]:
for ii in item.getObj('chem_comp_atom'):
    print(ii)

In [ ]:
for ii in item.getObj('chem_comp_bond'):
    print(ii)

In [ ]:
item.getObjNameList()